This script should be executed after creation of "ABNodesUK.csv" and when "Route Line Creator" finishes the job and a "Result" shapefile is put in the appropriate folder.

for example:

`D:/Projects/rlc/Result.shp`

In [1]:
import sys
import os
import sqlite3
from osgeo import gdal

print('Python v' + sys.version)
print('SQLite v' + sqlite3.sqlite_version)
print('OSGeo v' + gdal.__version__)

Python v3.9.2 (tags/v3.9.2:1a79785, Feb 19 2021, 13:44:55) [MSC v.1928 64 bit (AMD64)]
SQLite v3.34.0
OSGeo v3.2.1


### Please set the project folder path here:

In [ ]:
project = "D:/Routelines/"

Please check path to input SHP file and output SHP file in the next section!

In [ ]:
rlc_input_path = project + 'rlc/'

print(rlc_input_path)

rlc = rlc_input_path + "Result"
shp_path = project + "shp/"
shp_out = shp_path + "Routelines_final"

print (rlc)
print (shp_path)
print (shp_out)


#### RLC shapefile import into SQLite as a spatial table:

CREATE VIRTUAL TABLE Result USING VirtualShape("D:/Routelines/rlc/Result", "CP1251", 27700);

In [ ]:
def RLC_shp_import(rlc):
    print("Loading RLC shapefile ...")
    c.execute('''CREATE VIRTUAL TABLE Result USING VirtualShape("''' + rlc + '''", "CP1251", 27700);''')
    conn.commit()

`routes_geom()` will connect/merge all the lines that creates one route (with a unique BM_RouteID) 
based on the RoutelinesUK file where each route has N pairs of lines of which it consists.

_e.g. route with BM_RouteID "NW_SCMY_X2_O_354_430" consists of 106 lines from ABNodesUK_

In [ ]:
def routes_geom():
    print("Creating route geometries...")
    c.execute('''CREATE TABLE routes_geom(Region TEXT, BMRouteID TEXT, AnodeStopID TEXT, BnodeStopID TEXT);''')
    conn.commit()
    c.execute('''SELECT AddGeometryColumn('routes_geom', 'geometry', 27700, 'GEOMETRY', 'XY');''')
    conn.commit()
    c.execute('''INSERT INTO routes_geom (Region, BMRouteID, AnodeStopID, BnodeStopID, geometry) SELECT rl.Region, rl.BMRouteID, rl.AnodeStopID, rl.BnodeStopID, ST_Union(rs.geometry) FROM RoutelinesUK rl INNER JOIN Result rs ON rl.AnodeStopID = rs.astopid and rl.BnodeStopID = rs.bstopid GROUP BY BMRouteID;''')
    conn.commit()

`routelines_final()` will perform SQL JOIN based on BM_RouteID from the table "RLFreqUK" for the geometry / line of the complete route thus created

In [ ]:
def routelines_final():
    print("Joining route geometries with freq. data and creating final table...")
    c.execute('''CREATE TABLE Routelines_final(
                                               BM_RouteID TEXT, 
                                               OperatorCode TEXT, 
                                               ServiceNum TEXT, 
                                               Direction TEXT, 
                                               OperatorName TEXT,
                                               MonEarly REAL, MonAM REAL, MonBP REAL, MonEP REAL, MonOP REAL, MonNight REAL, 
                                               TueEarly REAL,TueAM REAL,  TueBP REAL, TueEP REAL, TueOP REAL, TueNight REAL,
                                               WedEarly REAL, WedAM REAL, WedBP REAL, WedEP REAL, WedOP REAL, WedNight REAL,
                                               ThurEarly REAL, ThurAM REAL, ThurBP REAL, ThurEP REAL, ThurOP REAL, ThurNight REAL,
                                               FriEarly REAL, FriAM REAL, FriBP REAL, FriEP REAL, FriOP REAL, FriNight REAL,
                                               SatEarly REAL, SatAM REAL, SatBP REAL, SatEP REAL, SatOP REAL, SatNight REAL,
                                               SunEarly REAL, SunAM REAL, SunBP REAL, SunEP REAL, SunOP REAL, SunNight REAL,
                                               id INT,
                                               pk_uid INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT);''')
    conn.commit()
    c.execute('''SELECT AddGeometryColumn('Routelines_final', 'geometry', 27700, 'GEOMETRY', 'XY');''')
    conn.commit()
    c.execute('''INSERT INTO Routelines_final (BM_RouteID,OperatorCode,ServiceNum,Direction,OperatorName,MonEarly,MonAM,MonBP,MonEP,MonOP,MonNight,TueEarly,TueAM,TueBP,TueEP,TueOP,TueNight,WedEarly,WedAM,WedBP,WedEP,WedOP,WedNight,ThurEarly,ThurAM,ThurBP,ThurEP,ThurOP,ThurNight,FriEarly,FriAM,FriBP,FriEP,FriOP,FriNight,SatEarly,SatAM,SatBP,SatEP,SatOP,SatNight,SunEarly,SunAM,SunBP,SunEP,SunOP,SunNight,id, geometry)
    SELECT rf.BM_RouteID, rf.OperatorCode, rf.ServiceNum, rf.Direction, rf.OperatorName, rf.MonEarly, rf.MonAM, rf.MonBP, rf.MonEP, rf.MonOP, rf.MonNight, rf.TueEarly, rf.TueAM, rf.TueBP, rf.TueEP, rf.TueOP, rf.TueNight, rf.WedEarly, rf.WedAM, rf.WedBP, rf.WedEP, rf.WedOP, rf.WedNight, rf.ThurEarly, rf.ThurAM, rf.ThurBP, rf.ThurEP, rf.ThurOP, rf.ThurNight, rf.FriEarly, rf.FriAM, rf.FriBP, rf.FriEP, rf.FriOP, rf.FriNight, rf.SatEarly, rf.SatAM, rf.SatBP, rf.SatEP, rf.SatOP, rf.SatNight, rf.SunEarly, rf.SunAM, rf.SunBP, rf.SunEP, rf.SunOP, rf.SunNight, rf.id, rg.geometry
    FROM RLFreqUK rf INNER JOIN routes_geom rg ON rf.BM_RouteID = rg.BMRouteID
    GROUP BY BMRouteID;''')
    conn.commit()

In [ ]:
def routelines_shp_export(db_table_name, geom_col, filename, encoding, geomtype):
    print("Exporting final table to shapefile...")
    sql = "SELECT ExportSHP('" + db_table_name + "', '" + geom_col + "', '" + filename + "', '" + encoding + "' , '" + geomtype + "')"
#     print(sql)
    c.execute(sql)
    conn.commit()

### Main routine

In [ ]:
# set the enivronment variable to enable "ExportSHP" function

os.environ['SPATIALITE_SECURITY'] = 'relaxed'

In [ ]:
# Connection to SQLite database 'routelines.db'

conn = sqlite3.connect(db_path + 'routelines.db')
conn.enable_load_extension(True)
conn.execute('SELECT load_extension("mod_spatialite")')   
conn.execute('SELECT InitSpatialMetaData(1);')
c = conn.cursor()

In [ ]:
RLC_shp_import(rlc)
routes_geom()
routelines_final()

# Export database table to shapefile:
routelines_shp_export("Routelines_final", "geometry", shp_out, "UTF-8", "LINESTRING")
print("Finished!")

In [ ]:
conn.close()
print("Connection to database closed!")

In [ ]:
# removing extra columns

ds = gdal.OpenEx(shp_out+".shp", gdal.OF_VECTOR | gdal.OF_UPDATE)
ds.ExecuteSQL("ALTER TABLE Routelines_final DROP COLUMN pk_uid")
ds.ExecuteSQL("ALTER TABLE Routelines_final DROP COLUMN id")

print("FINISHED !")